In [1]:
import sys
# sys.path.append('../')
# sys.path.append('../probcalkge')

In [5]:
from probcalkge import Experiment
from probcalkge import BetaCalibtator


In [2]:
import probcalkge

In [3]:
sys.path

['/home/ricky/TREAT/probcal-kge/notebooks',
 '/home/ricky/softwares/anaconda3/envs/ampligraph/lib/python37.zip',
 '/home/ricky/softwares/anaconda3/envs/ampligraph/lib/python3.7',
 '/home/ricky/softwares/anaconda3/envs/ampligraph/lib/python3.7/lib-dynload',
 '',
 '/home/ricky/softwares/anaconda3/envs/ampligraph/lib/python3.7/site-packages',
 '/home/ricky/softwares/anaconda3/envs/ampligraph/lib/python3.7/site-packages/IPython/extensions',
 '/home/ricky/.ipython',
 '../',
 '../probcalkge']